# 01 Work with Detectors

This notebook is all about how to get a detector to work with in simulation or data analysis. A detector can be generated by using a configuration. This configuration is loadable and saveable using the library [Pydantic](https://pydantic-docs.helpmanual.io/) which ensures solid typing, validation and serialization from and to json.

## Table of Contents

1. [Import Dependencies](#dependencies)
2. [Playing around with the configuration](#configuration)
3. [Creating a detector](#detector)
4. [Visualizing a detector](#visualization)

## Import dependencies <a class="anchor" id="dependencies"></a>

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np

## Playing around with the Configuration <a class="anchor" id="configuration"></a>

The configuration is written in a way that its easy to persist and pass around. Like that a detector can be recreated easily using the configuration. Later on when generating multiple datasets, having this interoperability will be crucial in making knowledge transferable.

Each configuration contains four parts:
* **geometry:** Contains the details on the detector's geometry (e.g. triangular)
* **string:** Contains the details on the properties for each string
* **module:** Contains the details on the properties for each module
* **pmt:** Contains the details on the properties for each pmt

In addition, the property `seed` can be set to ensure all random processes within the detector generation, like setting PMT noise rates can be fixed. Let's look at some examples:

In [2]:
from ananke.schemas.detector import StringConfiguration, PMTConfiguration, ModuleConfiguration, \
    LengthGeometryConfiguration, DetectorConfiguration, DetectorGeometries

string_configuration = StringConfiguration(
    z_offset=50.0,
    module_number=20,
    module_distance=50.0
)

pmt_configuration = PMTConfiguration(
    efficiency= 0.4,
    noise_rate = 1e-6,
    gamma_scale = .25
)

module_configuration = ModuleConfiguration(
    radius=20
)

geometry_configuration = LengthGeometryConfiguration(
    type=DetectorGeometries.TRIANGULAR,
    side_length=100
)

detector_configuration = DetectorConfiguration(
    geometry=geometry_configuration,
    string=string_configuration,
    module=module_configuration,
    pmt=pmt_configuration,
    seed=1337
)

print(detector_configuration)

geometry=LengthGeometryConfiguration(start_position=Position(x=0.0, y=0.0), type=<DetectorGeometries.TRIANGULAR: 'triangular'>, side_length=100) string=StringConfiguration(z_offset=50.0, module_number=20, module_distance=50.0) module=ModuleConfiguration(radius=20.0) pmt=PMTConfiguration(efficiency=0.4, noise_rate=1e-06) seed=1337


Tha looks fantastic, does not it. Only problem is that you have to create quite a lot of classes. Meet the power of pydantic. We can just create a dict which could be read via json or yaml and go from there. Is not that fantastic? As I am lazy, you can as well check out some default values

In [3]:
configuration = {
    "string": {
        "module_number": 20,
        "module_distance": 50
    },
    "pmt": {
        "efficiency": 0.4
    },
    "module": {
        "radius": 15
    },
    "geometry": {
        "type": "triangular",
        "side_length": 100,
    },
    "seed": 4545
}

detector_configuration_dict = DetectorConfiguration.parse_obj(configuration)

print(detector_configuration_dict)

geometry=LengthGeometryConfiguration(start_position=Position(x=0.0, y=0.0), type=<DetectorGeometries.TRIANGULAR: 'triangular'>, side_length=100) string=StringConfiguration(z_offset=0.0, module_number=20, module_distance=50.0) module=ModuleConfiguration(radius=15.0) pmt=PMTConfiguration(efficiency=0.4, noise_rate=0.00016) seed=4545


And it gets even better. This is totally error proof (as long as the code is correct in typing). Let's check it out!

In [4]:
configuration = {
    "string": {
        "module_number": 20
    },
    "pmt": {
        "efficiency": 0.4
    },
    "module": {
        "radius": 15
    },
    "geometry": {
        "type": "triangular",
        "side_length": 100,
    },
    "seed": 4545
}

detector_configuration_err = DetectorConfiguration.parse_obj(configuration)

print(detector_configuration_err)

ValidationError: 1 validation error for DetectorConfiguration
string -> module_distance
  field required (type=value_error.missing)

In [ ]:
configuration = {
    "string": {
        "module_number": "peter"
    },
    "pmt": {
        "efficiency": 5
    },
    "module": {
        "radius": -1
    },
    "geometry": {
        "type": "triangular",
        "number_of_strings_per_side": 45
    },
    "seed": 4545
}

detector_configuration_err = DetectorConfiguration.parse_obj(configuration)

Covering all possible errors here is impossible, but the idea should be clear right now.

## Creating a detector <a class="anchor" href="detector"></a>

Now we know how to handle the configurations. Let's create a detector. Wait for the magic:

In [42]:
from ananke.services.detector import DetectorBuilderService

configuration = {
    "string": {
        "module_number": 2,
        "module_distance": 50
    },
    "pmt": {
        "efficiency": 0.4
    },
    "module": {
        "radius": 15
    },
    "geometry": {
        "type": "triangular",
        "side_length": 100,
    },
    "seed": 4545
}

detector_configuration = DetectorConfiguration.parse_obj(configuration)

detector_service = DetectorBuilderService()

detector = detector_service.get(detector_configuration)

print(detector)

Detector(numpy_array=array([[-50.        , -28.86751346,   0.        , -50.        ,
        -28.86751346,   0.        , -50.        , -28.86751346,
          0.        ,   0.        ,   1.57079633,   0.        ,
          0.        ,   0.        ],
       [-50.        , -28.86751346,   0.        , -50.        ,
        -28.86751346,  50.        , -50.        , -28.86751346,
         50.        ,   0.        ,   1.57079633,   0.        ,
          1.        ,   0.        ],
       [ 50.        , -28.86751346,   0.        ,  50.        ,
        -28.86751346,   0.        ,  50.        , -28.86751346,
          0.        ,   0.        ,   1.57079633,   0.        ,
          0.        ,   1.        ],
       [ 50.        , -28.86751346,   0.        ,  50.        ,
        -28.86751346,  50.        ,  50.        , -28.86751346,
         50.        ,   0.        ,   1.57079633,   0.        ,
          1.        ,   1.        ],
       [  0.        ,  57.73502692,   0.        ,   0.        ,

That is how easy it gets :)